In [1]:
import grid2op
import numpy as np
from grid2op.Agent import DoNothingAgent

In [2]:
env = grid2op.make("rte_case14_redisp") # l2rpn_case14_sandbox
agent = DoNothingAgent(env.action_space)

obs = env.reset()
rew = env.reward_range[0]
done = False

In [3]:
env.gen_redispatchable

array([ True,  True, False, False,  True])

In [6]:
env.parameters.to_dict()

{'NO_OVERFLOW_DISCONNECTION': False,
 'IGNORE_MIN_UP_DOWN_TIME': True,
 'ALLOW_DISPATCH_GEN_SWITCH_OFF': True,
 'NB_TIMESTEP_OVERFLOW_ALLOWED': 2,
 'NB_TIMESTEP_RECONNECTION': 10,
 'HARD_OVERFLOW_THRESHOLD': 2.0,
 'ENV_DC': False,
 'FORECAST_DC': False,
 'MAX_SUB_CHANGED': 1,
 'MAX_LINE_STATUS_CHANGED': 1,
 'NB_TIMESTEP_COOLDOWN_LINE': 0,
 'NB_TIMESTEP_COOLDOWN_SUB': 0}

In [3]:
env.gen_cost_per_MW

array([40., 70.,  0.,  0.,  0., 70.], dtype=float32)

In [3]:
env.get_obs().target_dispatch

array([0., 0., 0., 0., 0.], dtype=float32)

In [6]:
env.n_gen

5

In [5]:
env.get_thermal_limit()

array([3.8490018e+02, 3.8490018e+02, 2.2899711e+05, 2.2899711e+05,
       2.2899711e+05, 1.5266474e+04, 2.2899711e+05, 3.8490018e+02,
       3.8490018e+02, 1.8328580e+02, 3.8490018e+02, 3.8490018e+02,
       2.2899711e+05, 2.2899711e+05, 6.9393062e+04, 3.8490018e+02,
       3.8490018e+02, 2.4056261e+02, 3.4022427e+03, 3.4022427e+03],
      dtype=float32)

In [15]:
print(env.backend.get_line_overflow())
print(env.backend.get_relative_flow())
obss = env.get_obs()
print(500**np.array(obss.timestep_overflow) - 1)

[False False False False False False False False False False False False
 False False False False False False False False]
[0.4463135  0.39749488 0.1377533  0.1745734  0.09882749 0.32277596
 0.09506696 0.2714448  0.47793412 0.8714953  0.3802412  0.36823758
 0.12878639 0.10744023 0.83211094 0.31076756 0.19732746 0.8781536
 0.08645719 0.373654  ]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [27]:
print(obs.minute_of_hour)
print(obs.hour_of_day)
print(obs.day)
print(obs.day_of_week)
print(obs.month)
print(obs.year)

0
0
6
6
1
2019


In [34]:
obs.timestep_overflow

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [39]:
# input to NN
obs.prod_p / obs.gen_pmax

array([0.504     , 0.3685    , 0.38142857, 0.        , 0.24034223],
      dtype=float32)

In [49]:
obs.rho

array([0.4463135 , 0.39749488, 0.1377533 , 0.1745734 , 0.09882749,
       0.32277596, 0.09506696, 0.2714448 , 0.47793412, 0.8714953 ,
       0.3802412 , 0.36823758, 0.12878639, 0.10744023, 0.83211094,
       0.31076756, 0.19732746, 0.8781536 , 0.08645719, 0.373654  ],
      dtype=float32)

In [50]:
# input to NN
limit = 2 #200% of power line disconnects automatically
(np.sign(obs.p_or) * obs.rho - (-limit)) / (limit - (-limit))

array([0.61157835, 0.5993737 , 0.5344383 , 0.54364336, 0.47529313,
       0.58069396, 0.52376676, 0.5678612 , 0.61948353, 0.7178738 ,
       0.5950603 , 0.4079406 , 0.5321966 , 0.52686006, 0.7080277 ,
       0.5776919 , 0.54933184, 0.7195384 , 0.4783857 , 0.5934135 ],
      dtype=float32)

In [51]:
obs.attr_list_vect

['year',
 'month',
 'day',
 'hour_of_day',
 'minute_of_hour',
 'day_of_week',
 'prod_p',
 'prod_q',
 'prod_v',
 'load_p',
 'load_q',
 'load_v',
 'p_or',
 'q_or',
 'v_or',
 'a_or',
 'p_ex',
 'q_ex',
 'v_ex',
 'a_ex',
 'rho',
 'line_status',
 'timestep_overflow',
 'topo_vect',
 'time_before_cooldown_line',
 'time_before_cooldown_sub',
 'time_next_maintenance',
 'duration_next_maintenance',
 'target_dispatch',
 'actual_dispatch']

In [20]:
import numpy as np
print(obs.name_line)
print(obs.line_ex_to_subid)
print(obs.line_or_to_subid)
print('-'.join(int(obs.line_status)))

['0_1_0' '0_4_1' '8_9_2' '8_13_3' '9_10_4' '11_12_5' '12_13_6' '1_2_7'
 '1_3_8' '1_4_9' '2_3_10' '3_4_11' '5_10_12' '5_11_13' '5_12_14' '3_6_15'
 '3_8_16' '4_5_17' '6_7_18' '6_8_19']
[ 1  4  9 13 10 12 13  2  3  4  3  4 10 11 12  6  8  5  7  8]
[ 0  0  8  8  9 11 12  1  1  1  2  3  5  5  5  3  3  4  6  6]


TypeError: only size-1 arrays can be converted to Python scalars

In [26]:
obs.rho > 0.8

array([False, False, False, False, False, False, False, False, False,
        True, False, False, False, False,  True, False, False,  True,
       False, False])

In [21]:
np.where(obs.rho > 0.8)

(array([ 9, 14, 17], dtype=int64),)

In [27]:
np.argmax(obs.rho)

17

In [42]:
sum(obs.gen_redispatchable)

3

In [30]:
print("from {} to {}".format(obs.line_or_to_subid[17], obs.line_ex_to_subid[17]))

from 4 to 5


In [4]:
obs.prod_p

array([75.6    , 73.7    , 26.7    ,  0.     , 72.10267], dtype=float32)

In [5]:
#act = agent.act(obs, rew, done)
#obs, rew, done, info = env.step(act)

In [6]:
obs.line_status

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [7]:
obs.actual_dispatch

array([0., 0., 0., 0., 0.], dtype=float32)

In [11]:
obs.gen_to_subid

array([1, 2, 5, 7, 0])

In [12]:
obs.p_or

array([ 3.9404995e+01,  3.2697674e+01,  2.5208673e+00,  7.8256321e+00,
       -6.5202169e+00,  1.6684830e+00,  7.3431721e+00,  2.4547935e+01,
        3.8133049e+01,  3.0225504e+01,  1.5380751e+01, -3.2670959e+01,
        1.0081982e+01,  7.4650497e+00,  1.7967918e+01,  2.4900978e+01,
        1.4345522e+01,  2.1414949e+01, -2.9087843e-14,  2.4900978e+01],
      dtype=float32)

In [13]:
obs.load_p

array([21.8, 82.6, 14.8, 45. ,  7.1, 12.6, 28.9,  9. ,  3.4,  5.7, 12. ],
      dtype=float32)

In [18]:
obs.gen_max_ramp_up

array([ 5., 10.,  0.,  0., 10.], dtype=float32)

In [19]:
obs.gen_max_ramp_down

array([ 5., 10.,  0.,  0., 10.], dtype=float32)

In [37]:
np.array(range(obs.n_gen))[obs.gen_redispatchable]

array([0, 1, 4])

In [50]:
print(obs.gen_to_subid)
for i, g in enumerate(obs.prod_p):
    print(i)
    print(g)

[1 2 5 7 0]
0
75.6
1
73.7
2
26.7
3
0.0
4
72.10267


In [44]:
np.array(range(obs.n_sub))

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [7]:
res_dict = {}
gen_cont = {}
redis_gen_ids = [0,1,4]
gen_max_ramp_down = [10, 5, 0, 0, 10]
gen_max_ramp_up = [10, 5, 0, 0, 10]

res_dict = {}
gen_action_ixs = [0 for i in range(3)]
for i in range(3**3):
    res_dict[i] = _add_group_action(gen_action_ixs)
    gen_action_ixs = _change_action_ixs(i, gen_action_ixs)

print(res_dict)

{0: [(0, -10), (1, -5), (4, -10)], 1: [(0, 0), (1, -5), (4, -10)], 2: [(0, 10), (1, -5), (4, -10)], 3: [(0, -10), (1, 0), (4, -10)], 4: [(0, 0), (1, 0), (4, -10)], 5: [(0, 10), (1, 0), (4, -10)], 6: [(0, -10), (1, 5), (4, -10)], 7: [(0, 0), (1, 5), (4, -10)], 8: [(0, 10), (1, 5), (4, -10)], 9: [(0, -10), (1, -5), (4, 0)], 10: [(0, 0), (1, -5), (4, 0)], 11: [(0, 10), (1, -5), (4, 0)], 12: [(0, -10), (1, 0), (4, 0)], 13: [(0, 0), (1, 0), (4, 0)], 14: [(0, 10), (1, 0), (4, 0)], 15: [(0, -10), (1, 5), (4, 0)], 16: [(0, 0), (1, 5), (4, 0)], 17: [(0, 10), (1, 5), (4, 0)], 18: [(0, -10), (1, -5), (4, 10)], 19: [(0, 0), (1, -5), (4, 10)], 20: [(0, 10), (1, -5), (4, 10)], 21: [(0, -10), (1, 0), (4, 10)], 22: [(0, 0), (1, 0), (4, 10)], 23: [(0, 10), (1, 0), (4, 10)], 24: [(0, -10), (1, 5), (4, 10)], 25: [(0, 0), (1, 5), (4, 10)], 26: [(0, 10), (1, 5), (4, 10)]}


In [2]:
def _add_group_action(gen_action):
    one_action = []
    for i, g_id in enumerate(redis_gen_ids):
        act_g = _get_gen_action(g_id, gen_action[i])
        one_action.append((g_id, act_g))
    return one_action

In [1]:
def _get_gen_action(gen_id, action_ix):
    """
    Define the granularity of actions. Intermediated values could be added.
    If add here, change parameter self.ACTIONS_PER_GEN
    """
    if action_ix == 0:
        return -gen_max_ramp_down[gen_id]
    elif action_ix == 1:
        return 0
    elif action_ix == 2:
        return gen_max_ramp_up[gen_id]
    else:
        raise Exception("Action index out of bounds")

In [3]:
def _change_action_ixs(act_ix, gen_action_ixs):
    """
    cont_g_ix tells which generator is changing.
    gen_action_ixs contains action index for each generator.
    """
    limits = [3 ** i for i in range(3)]
    # [1, 3, 9]
    
    for ix, gen_lim in enumerate(limits):
        if (act_ix + 1) % gen_lim == 0:
            if gen_action_ixs[ix] == (3 - 1):
                gen_action_ixs[ix] = 0
            else:
                gen_action_ixs[ix] += 1

    return gen_action_ixs